考虑到实变函数与泛函分析（所谓的现代分析基础）的课程十分困难而且临近考试，这个大致做一个骨架就行了，其内容应足以支持
所谓的20页ppt与5000字报告的无理要求。

## 为何把查重看成二分类这一不合理的思路有一些合理性

抄袭分类本身本来不应当看成纯粹的分类问题，两个十分相似的文档经因为先后顺序原因被分为抄袭与不抄袭，是与文档内容无关的信息。
不过像课程报告之类的东西，大多数人应该态度并不那么严谨，这就使得他们本人应该写出来的东西和他们抄袭的东西的风格不一致。
这个蹩脚的理由似乎可以为我使用纯分类模型做辩护（如果时间够我倒宁愿做一个考虑先后顺序的变分推断模型，作为我本科论文的扩展）。

## 经典方法

NLP的文档分类的老方法主要是特征工程化的，比如文档可以被一个词袋模型的计数向量所“刻画”，这种方法风行于计算机性能差劲的时代。甚至
图像识别也取一些patch算相关来算类似词袋的东西。

词袋之类的东西无非是一种向量化方法，也可以对整个文档直接对字母/汉字或词做One hot编码，这个东西太长了，整个一起处理性能不好
以及长度也不是固定的。不过在生成里有应用（几年前做了个demo https://github.com/yiyuezhuo/chinese_text_generator ）， 不过end to end流行以后这种轻特征工程法也越来越多了。

对词啊，字符啊做一些无监督的，与之后的分类器无关的处理看起来是有用的，也很适合灌水，这就整出了ID-IDF这种对词袋的文档-词频矩阵的改进方法。它
给鉴别力强的词加权，根据运行它时喂给它的一堆文档，可以看作一种对分布的轻量级建模，从而这种方法碰到分布变化时自然失败（就像
所有机器学习算法一样）。几年前我上毛概的时候也用此算法运行到人民日报的文章上把TG批判了一番： https://github.com/yiyuezhuo/party-text-research

文档-词频以及变体继续发展就得到了主题模型。潜在语义分析LSA(Latent semantic analysis)利用类似主成分分析的方法以SVD分解出主题。
让我想起我上多元统计课时那书对如何解释主成分的含义大伤脑筋。PCA由于有SPSS之类的傻瓜软件的加持，在伪统计学流行的领域变成一股泥石流。
类似的，LDA(Latent Dirichlet allocation)似乎也造出一群不懂生成模型意义的人的滥用。我在暑假时候和一个人谈论ARMA就深感似乎有些人
受到确定性模型的毒害，根本无法理解甚至像ARMA模型这么简单的生成模型与概率模型的原理。

总之，有别于LDA之类的将文档转成一个定长向量的无监督方法。从one hot上发展出词向量法。考虑到one hot编码，特别是对词向量来说维度太高。
可以考虑按词的“意义”把它们降维，我在写这段之前发现了这个中文词向量的数据（ https://github.com/Embedding/Chinese-Word-Vectors ）
把几十万维的东西降成300维看起来十分有效。它所用的word2vec算法和很多迁移学习法一样，就是把一个整体的一部分切下来单独用。
在图像分类中把Alexnet之类的5层网络（那两三年的论文好像对5层特别热衷）单独拿出来，fine tune后面的部分。或者分类的拿出来，fine tune
成object detection的网络。或者自编码器或VAE或GAN拿出生成的那一部分一样。

词向量对应的不定长的文档矩阵没法直接套用SVM或最后放个SVM的神经网络，所以处理它们最开始用RNN和LSTM之类的网络，后来CNN在图像中
十分流行导致CNN也用到文本上了（考虑到有几种CNN专门处理过变尺寸图片的问题，这也不是不能理解。）。而图形的ResNet之类的乱连的网络
又激发了NLP的attention的想法。然后attention又被改造了CV的算法。。

我这里想用CNN分类文档，因为我之前看了一些CNN的论文，不过图像的训练资源要求太高，先试试文本的（虽然此时我还没拿到数据，
但估计最多几百个文档，对我这个既没有教研室个人电脑，组里也没有服务器而只有垃圾笔记本来说十分合适）。

## CNN如何分类文档

CNN在图像上有一些网络最后一层可以生成变大小的feature map。这个东西可以看成那个格子上对应receptive field在CNN眼里应该怎么分类。
在前面几层中，则可以看成某个滤波器对应特征在receptive field的激发大小。如一个点值很大可能暗示它的receptive field中有一个脸
的模式。

这类网络用在文本上也十分直接，它中间的层的响应可以看成某个特定文本模式，如骂人的激发程度。它最后一层可以看成不同receptive field的
管中窥豹的分类。如这段也许几百的词的话是否在谈论关于蛤蟆的事情。但对我这个问题，虽然现在我还没看到数据，但receptive field必然不能覆盖
整个文档。那么它所看到的“一段”文档的意义是什么呢？如果一个人只能看到一篇5000字文章的50字段落，他应该能鉴别出这篇文章是否是抄袭的吗？
如果人都不能做到，算法做到是否只是一种对噪声细节的滥用的过拟合？

图像的CNN的变长输出主要用于object detection，也用来引入一些偏置改善分类性能。对于物体尺寸大小则可能使用经典的金字塔法，或者spp-net
那种金字塔蕴含在网络内部的方法。金字塔的缩放本身也用来数据加强改善robustness。但文本上运用金字塔看上去没什么意义，不然就会看到“what 
what is is your your name name ? ?”之类的东西。和图像经常因为各种原因被缩放不同，文本的长度则更有意义。